# Sistema de recomendações por filtragem colaborativa

O dataset usado pode ser baixado no link a seguir:

https://drive.google.com/file/d/1EaYcOP1jhiOPtXkSHS4w1AIlk_UuwWLK/view?usp=sharing

# Importação de bibliotecas e módulos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from collections import Counter

# Leitura do dataset

In [2]:
data = pd.read_parquet("/content/pinterest.parquet")
display(data)

,user_id,item_id,rating
0,U-0,I-2,1
1,U-0,I-3,1
2,U-0,I-4,1
3,U-0,I-5,1
4,U-0,I-6,1
...,...,...,...
1445617,U-55186,I-5448,1
1445618,U-55186,I-4615,1
1445619,U-55186,I-5346,1
1445620,U-55186,I-2803,1


# Remoção de duplicatas

In [3]:
data.drop_duplicates(inplace=True)
display(data)

,user_id,item_id,rating
0,U-0,I-2,1
1,U-0,I-3,1
2,U-0,I-4,1
3,U-0,I-5,1
4,U-0,I-6,1
...,...,...,...
1445617,U-55186,I-5448,1
1445618,U-55186,I-4615,1
1445619,U-55186,I-5346,1
1445620,U-55186,I-2803,1


# Calculando o número de usuários, itens e avaliações

In [4]:
n_users = data['user_id'].nunique()
n_itens = data['item_id'].nunique()
n_ratings = data[['user_id','item_id']].shape[0]

print(f'O número de itens do dataset é {n_itens}.')
print(f'O número de usuários do dataset é {n_users}.')
print(f'O número de avaliações do dataset é {n_ratings}.')

O número de itens do dataset é 9911.
O número de usuários do dataset é 55187.
O número de avaliações do dataset é 1408394.


# Criando grafos

In [5]:
G = nx.Graph()
G.add_nodes_from(data['item_id'].unique(),node_type='item')
G.add_nodes_from(data['user_id'].unique(),node_type='user')
G.add_weighted_edges_from(data[['user_id','item_id','rating']].values)
G

print(f'O grafo possui {G.number_of_nodes()} nós e {G.number_of_edges()} arestas.')



O grafo possui 65098 nós e 1408394 arestas.


# Fazendo o exemplo de um item

In [6]:
neighbors = list(G.neighbors('I-5448'))
neighbors_consumed_items = []
for x in neighbors:
  neighbors_consumed_items += list(G.neighbors(x))

df = dict(Counter(neighbors_consumed_items))
df = pd.DataFrame([list(df.keys()),list(df.values())])
df = df.T
df.columns = ['item_id','score']
df = df.sort_values(by='score',ascending=False)

y = ','.join(neighbors_consumed_items[0:10])
print(f'Quem viu I-5448 também viu: {y}.')

display(df)

Quem viu I-5448 também viu: I-5186,I-4665,I-2937,I-1751,I-3010,I-5445,I-5446,I-5447,I-5444,I-5237.


,item_id,score
19,I-5448,95
548,I-5185,9
329,I-7024,8
389,I-6891,7
479,I-1548,7
...,...,...
635,I-3216,1
636,I-1477,1
637,I-1428,1
638,I-6104,1


# Criando uma função para achar as recomendações para um determinado item

In [7]:
def recomended_items(G:'grafo de itens e usuários',item_id:'item',n:'number of recomendations'):
  node_type = nx.get_node_attributes(G,'node_type')[item_id]
  if(node_type != 'item'):
    raise ValueError('Node is not of "item" type!')

  neighbors = list(G.neighbors(item_id))
  neighbors_consumed_items = []
  for x in neighbors:
    neighbors_consumed_items += list(G.neighbors(x))

  dicio = dict(Counter(neighbors_consumed_items))

  df = pd.DataFrame([list(dicio.keys()),list(dicio.values())])
  df = df.T
  df.columns = ['item_id','score']
  df = df.sort_values(by='score',ascending=False)
  df = df[0:n].reset_index(drop=True)

  return df

# Aplicando a função para achar os 10 itens mais recomendados para um certo usuário

In [8]:
recomended_items(G,'I-5448',10)


,item_id,score
0,I-5448,95
1,I-5185,9
2,I-7024,8
3,I-6891,7
4,I-1548,7
5,I-154,7
6,I-4091,7
7,I-5427,7
8,I-3715,6
9,I-1485,6
